In [1]:
# ms-python.python added
import os
'''
try:
	os.chdir(os.path.join(os.getcwd(), '../../../../../../var/folders/lx/703nk8wx7vb585ttwyxk_1lr0000gn/T'))
	print(os.getcwd())
except:
	pass
'''

"\ntry:\n\tos.chdir(os.path.join(os.getcwd(), '../../../../../../var/folders/lx/703nk8wx7vb585ttwyxk_1lr0000gn/T'))\n\tprint(os.getcwd())\nexcept:\n\tpass\n"

In [2]:
from IPython import get_ipython


 # Training Model
 Keras on Tensorflow


In [3]:
# import os, glob, shutil
import os, sys
import cv2
import numpy as np
import uuid
import tensorflow as tf
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from imutils import paths
import itertools

In [4]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from core.imageprep import random_crop, crop_generator, random_crop_batch
from core.models import UNet


get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [5]:
%load_ext tensorboard

 ## Load Training Dataset

In [6]:
# load image
print("Load Images...")
# on mac
path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
# path = "D:/PerlmutterData/"
imgdir = "training/cell_membrane/prepdata"
imgpath = os.path.join(path, imgdir)


Load Images...


 Print the first file name.

In [7]:
imgpath_all = list(paths.list_images(imgpath))
print(imgpath_all[0])


/Volumes/LaCie_DataStorage/PerlmutterData/training/cell_membrane/prepdata/train/images/cell_membrane/0001.tif


 ## Set Parameters

In [8]:
# set parameters
seed = 100
batch_size = 16
validation_split = 0.1
training_sample_size = len(imgpath_all)
IMG_HEIGHT = 256
IMG_WIDTH = 256


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [9]:
# create arguments for data generator
data_gen_args = dict(
                featurewise_center=True,
                featurewise_std_normalization=True,
                horizontal_flip=True,
                vertical_flip=True,
                rotation_range=90.,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.07,
                zoom_range=0.2,
                validation_split=validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                )



In [10]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_args)
label_datagen = ImageDataGenerator(**data_gen_args)

In [11]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    os.path.join(imgpath, 'train/images/'),
    class_mode=None,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # <- define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    os.path.join(imgpath, 'train/labels'),
    class_mode=None,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    os.path.join(imgpath, 'train/images/'),
    class_mode=None,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # <- define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    os.path.join(imgpath, 'train/labels'),
    class_mode=None,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)



Found 1800 images belonging to 1 classes.
Found 1800 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.


In [12]:
# merge image and label generator
train_generator = zip(train_image_generator, train_label_generator)
valid_generator = zip(valid_image_generator, valid_label_generator)


 ## Training

In [13]:
print("Start training...")



Start training...


In [14]:
# training
# load dependency
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from datetime import datetime


 ### Define checkpointer

In [15]:
# checkpointer
# check if the output folder exist. If not, create a folder
if not 'model' in os.listdir(path):
    os.mkdir(os.path.join(path, 'model'))

# set up the checkpointer
# save model
modelfilename = 'model_' + datetime.now().strftime("%Y_%m_%d_%H_%M") + '.h5'
modelfilepath = os.path.join(path, 'model', modelfilename)

checkpointer = ModelCheckpoint(modelfilepath, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

log_dir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


 ### Setup the model

In [16]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1-validation_split) // batch_size
print("Steps per epoch: {}".format(steps_per_epoch))


Steps per epoch: 225.0


In [17]:
# prepare the model
unetmodel = UNet([IMG_HEIGHT, IMG_WIDTH])

# train the model
unetmodel.fit_generator(generator=train_generator, 
                    validation_data = valid_generator,
                    validation_steps = 20,
                    steps_per_epoch = steps_per_epoch,
                    epochs = 500, 
                    verbose=1, 
                    callbacks=[checkpointer], 
                    # callbacks=[tensorboard_callback]
                    )


conv1 shape: (None, 256, 256, 64)
conv1 shape: (None, 256, 256, 64)
pool1 shape: (None, 128, 128, 64)
conv2 shape: (None, 128, 128, 128)
conv2 shape: (None, 128, 128, 128)
pool2 shape: (None, 64, 64, 128)
conv3 shape: (None, 64, 64, 256)
conv3 shape: (None, 64, 64, 256)
pool3 shape: (None, 32, 32, 256)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]        

AttributeError: 'zip' object has no attribute 'shape'

In [ ]:
%tensorboard --logdir logs/fit